In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn

In [13]:
def getfiles(start="bhist",items='120',design='v30',k='k10',schemes=['fixed_express'],mis=0):
    listoffiles=[]
    where='Results/'
    kmap=getkmap()
    for file in os.listdir(where):
        if file.startswith(start):
            num=[x for x in schemes if x in file]
            if items in file and design in file and len(num)>0 and (file.endswith('mis.csv')==mis):
                if ('test' not in file) and ('more' not in file):
                    if kmap[file.split('-')[1]]:
                        if k in file:
                            #listoffiles.append(file)
                            if file.endswith(k+'.csv'):
                                listoffiles.append(file)
                    else:
                        #listoffiles.append(file)
                        if file.endswith(design+'.csv'):
                            listoffiles.append(file)
            #elif items in file and design in file and len(num)>0 and (2*file.endswith('moremis.csv')==mis):
            #    if kmap[file.split('-')[1]]:
            #        if k in file:
            #            listoffiles.append(file)
                        #if file.endswith(k+'.csv'):
                        #    listoffiles.append(file)
            #    else:
            #        listoffiles.append(file)
                
    return where,listoffiles

In [2]:
def getcolormap():
    colormap={'TS':'Navy','TSe4':'Aqua','TSe1':'Red','TSe2':'Maroon','TSe3':'Aqua','TSthres':'Blue','TSdouble':'Navy',
              'greedy':'Green', 'greedy3':'Green','greedythres':'Green',
              'fixed_express':'Fuchsia','fixed_sparse':'Purple','uncert':'Red','mismin':'Maroon',
             'TSe4apprx':'Fuchsia','TSapprox':'Fuchsia','TSe3apprx':'Purple','TSthresapprx':'Purple',
              'TSapproxthres':'Purple','misminapprox':'Green','winmismin':'Red',
              'TSregthres':'Teal','newTSthres':'Blue','newTSregthres':'Teal','greedynightmare':'Lime',
              'winapprox':'Green','winapproxthres':'Lime',
             'TSe1d2':'Red','TSe1d4':'Maroon','TSe4d0':'Green','TSe2d0':'Green','TSe2d2':'Purple',
              'TSe4d2':'Purple','TSe2d4':'Aqua'}#:'Teal'
    return colormap

In [3]:
def getkmap():
    kmap={'TS':0,'TSe4':0,'TSe1':0,'TSe2':0,'TSthres':1,'TSdouble':1, 'greedy':0,'greedythres':1,
              'fixed_express':0,'fixed_sparse':0,'uncert':1,'mismin':1,
         'TSe4apprx':0,'TSapprox':0,'TSthresapprx':1,'TSapproxthres':1,'misminapprox':1,'TSregthres':1,
         'newTSthres':1,'newTSregthres':1,'TSe3apprx':0,'TSe3':0,'greedy3':0,'greedynightmare':0,
          'winapprox':0,'winapproxthres':1,'TSe1d2':0,'TSe1d4':0,'TSe4d0':0,
          'TSe2d0':0,'TSe2d2':0,'TSe4d2':0,'TSe2d4':0,'winmismin':1}
    return kmap

In [4]:
def getlinemap():
    linemap={'TS':'solid','TSe4':'dashed','TSe1':'dashdot','TSe2':'dotted','TSe3':'dotted' 
             ,'TSthres':'dotted' ,'TSdouble':'dotted' ,'winmismin':'solid',
              'greedy':'dashdot' , 'greedy3':'dotted' ,'greedythres':'dashdot' ,
              'fixed_express':'dotted','fixed_sparse':'dotted' ,'uncert':'solid','mismin':'dashed',
             'TSe4apprx':'dotted','TSe3apprx':'dotted','TSthresapprx':'dotted','misminapprox':'dotted',
              'TSregthres':'dashdot',
             'TSapprox':'dotted','TSapproxthres':'dotted',
             'newTSthres':'dashdot','newTSregthres':'dashdot',
             'greedynightmare':'dotted',
              'winapprox':'solid','winapproxthres':'dashdot',
             'TSe1d2':'dotted','TSe1d4':'solid','TSe4d0':'dotted','TSe2d0':'dotted','TSe2d2':'dashdot',
              'TSe4d2':'dashdot','TSe2d4':'dashed'}#:'Teal'
    return linemap

In [5]:
def getlinemap2():
    linemap={'TS':'solid','TSe4':'dashed','TSe1':'dashdot','TSe2':'dotted','TSe3':'dotted' 
             ,'TSthres':'dotted' ,'TSdouble':'dotted' ,'winmismin':'solid',
              'greedy':'dashdot' , 'greedy3':'dotted' ,'greedythres':'dashdot' ,
              'fixed_express':'solid','fixed_sparse':'dotted' ,'uncert':'solid','mismin':'dashed',
             'TSe4apprx':'dotted','TSe3apprx':'dotted','TSthresapprx':'dotted','misminapprox':'dotted',
              'TSregthres':'dashdot',
             'TSapprox':'dotted','TSapproxthres':'dotted',
             'newTSthres':'dashdot','newTSregthres':'dashdot',
             'greedynightmare':'dotted',
              'winapprox':'solid','winapproxthres':'dashdot',
             'TSe1d2':'dotted','TSe1d4':'solid','TSe4d0':'dotted','TSe2d0':'dotted','TSe2d2':'dashdot',
              'TSe4d2':'dashdot','TSe2d4':'dashed'}#:'Teal'
    return linemap

In [6]:
def getlabelmap():
    labelmap={'TS':'TS','TSe4':r'$\epsilon$-$\delta$-diffuse TS ($\epsilon=\frac{1}{4}$,$\delta=\frac{1}{4}$)',
              'TSe1':'dashdot','TSe2':'dotted','TSe3':'dotted' 
             ,'TSthres':'dotted' ,'TSdouble':'dotted' ,
              'greedy':'$\epsilon$-greedy' , 'greedy3':'dotted' ,
              'greedythres':'$\epsilon$-greedy with thresholding' ,
              'fixed_express':'Express','fixed_sparse':'dotted' ,
              'uncert':'max-uncert',
              'mismin':'max-misclass',
              'TSapprox':'TSapprox','TSapproxthres':'TSapproxthres',
             'TSe4apprx':'dotted','TSe3apprx':'dotted','TSthresapprx':'dotted','misminapprox':'misminapprox',
              'TSregthres':'dotted','newTSthres':'$\epsilon$-$\delta$-diffuse TS with thresholding',
              'newTSregthres':'TS with thresholding',
              'greedynightmare':'dotted',
              'winapprox':'WinApprox','winapproxthres':'WinApprox with thresholding',
             'TSe1d2':'dotted','TSe1d4':'solid','winmismin':'winmismin',
              'TSe4d0':r'$\epsilon$-$\delta$-diffuse TS ($\epsilon=\frac{1}{4}$,$\delta=0$)',
              'TSe2d0':r'$\epsilon$-$\delta$-diffuse TS ($\epsilon=\frac{1}{2}$,$\delta=0$)',
              'TSe2d2':r'$\epsilon$-$\delta$-diffuse TS ($\epsilon=\frac{1}{2}$,$\delta=\frac{1}{2}$)',
              'TSe4d2':r'$\epsilon$-$\delta$-diffuse TS ($\epsilon=\frac{1}{4}$,$\delta=\frac{1}{2}$)',
              'TSe2d4':r'$\epsilon$-$\delta$-diffuse TS ($\epsilon=\frac{1}{2}$,$\delta=\frac{1}{4}$)'}#:'Teal'
    return labelmap

In [7]:
def histplotsshort(items='120',k=10,design='v30',schemes=['fixed_express'],name='name.pdf',ylim=[.5,1],start=2,end=24,mean=1,iters=100,mis=0,low=-1):
    where,listoffiles=getfiles(start='bhist',items=items,k='k'+str(k),design=design,schemes=schemes,mis=mis)
    print(listoffiles)
    if items=='300':
        itemfile='HB_300items.csv'
        nitems=300
    elif items=='40':
        itemfile='HB_40items.csv'
        nitems=40
    elif items=='35':
        itemfile='ZB_35items.csv'
        nitems=35
    else:
        itemfile='HB_120items.csv'
        nitems=120
    data1=pd.read_csv(itemfile)
    utilsall=data1.get_values()[:,2:]
    realmean=np.mean(utilsall,0)
    if low<0:
        low=k
    topk=np.argsort(realmean)[::-1][:k]
    itemcol=[]
    for i in range(1,nitems+1):
        itemcol+=['Item_'+str(i)]
    resps=50
    topresp=np.zeros((iters,resps))
    resp=[x for x in range(20,1020,20)]
    #resp=[x for x in range(10,520,10)]
    colormap=getcolormap()
    linemap=getlinemap()
    labelmap=getlabelmap()
    for file in listoffiles:
            data=pd.read_csv(where+'/'+file)
            label=file.split('-')[1]
            for j in range(iters):
                
                iterdata=data[data['Iter']==(j+1)]
                for i,cur in enumerate(np.argsort(iterdata[itemcol].get_values())[:,::-1][:,:low]):
                    topresp[j,i]=(len(set.intersection(set(cur),set(topk))))/k
            if mean==1:
                topkmean=np.mean(topresp,0)
            else:
                topkmean=np.var(topresp,0)
            #topk95=np.percentile(topresp, 90,0)
            #topk5=np.percentile(topresp, 10,0)
            c=colormap[label]
            l=linemap[label]
            lab=labelmap[label]
            cur=plt.plot(resp[start:end+1],topkmean[start:end+1],label=lab,color=c,linestyle=l)
            #plt.fill_between(resp[start:end+1], topk5[start:end+1], topk95[start:end+1],
            #                 facecolor=colors[z], alpha=0.5)
    plt.xlabel('Respondent')
    
    if mean==1:
        if mis:
            plt.legend(loc='upper left',fontsize='small')
        else:
            plt.legend(loc='lower right',fontsize='small')
        plt.ylabel('Hit Rate')
        if low==k:
            plt.title('Top '+str(k)+' Hit Rate Over Respondents')
        else:
            plt.title('Top '+str(k)+' in the Top '+str(low)+' Hit Rate')
    else:
        plt.legend(loc='upper right',fontsize='small')
        plt.ylabel('Var')
        plt.title('Top '+str(k)+' Variance Over Respondents')
    plt.ylim(ylim)
    plt.xlim([resp[start],resp[end]])
    plt.savefig('plots/'+name, dpi=100)
    plt.close()

In [8]:
def histplotsPTUshort(items='120',k=10,design='v30',schemes=['fixed_express'],name='name.pdf',start=4,end=24,mean=1,mis=0):
    if items=='300':
        itemfile='HB_300items.csv'
        nitems=300
    else:
        itemfile='HB_120items.csv'
        nitems=120
    where,listoffiles=getfiles(start='bhist',items=items,k='k'+str(k),design=design,schemes=schemes,mis=mis)
    print(listoffiles)
    data1=pd.read_csv(itemfile)
    utilsall=data1.get_values()[:,2:]
    realmean=np.mean(utilsall,0)
    topk=np.argsort(realmean)[::-1][:k]
    total=np.sum(np.exp(realmean[topk]))
    
    itemcol=[]
    for i in range(1,nitems+1):
        itemcol+=['Item_'+str(i)]
    resps=50
    iters=100
    topresp=np.zeros((iters,resps))
    resp=[x for x in range(20,1020,20)]
    colormap=getcolormap()
    linemap=getlinemap()
    labelmap=getlabelmap()
    for file in listoffiles:
            data=pd.read_csv(where+'/'+file)
            label=file.split('-')[1]
            for j in range(iters):
                
                iterdata=data[data['Iter']==(j+1)]
                for i,cur in enumerate(np.argsort(iterdata[itemcol].get_values())[:,::-1][:,:k]):
                    topresp[j,i]=np.sum(np.exp(realmean[cur]))/total
            if mean==1:
                topkmean=np.mean(topresp,0)
            else:
                topkmean=np.var(topresp,0)
            #topk95=np.percentile(topresp, 90,0)
            #topk5=np.percentile(topresp, 10,0)
            c=colormap[label]
            l=linemap[label]
            lab=labelmap[label]
            cur=plt.plot(resp[start:end+1],topkmean[start:end+1],label=lab,color=c,linestyle=l)
            #plt.fill_between(resp[start:end+1], topk5[start:end+1], topk95[start:end+1],
            #                 facecolor=colors[z], alpha=0.5)
    plt.xlabel('Respondent')
    
    if mean==1:
        plt.legend(loc='lower right',fontsize='small')
        plt.ylabel('PTU')
        plt.title('Top '+str(k)+' PTU Over Respondents')
    else:
        plt.legend(loc='upper right',fontsize='small')
        plt.ylabel('Var')
        plt.title('Top '+str(k)+' Variance (PTU) Over Respondents')
    
    plt.xlim([resp[start],resp[end]])
    plt.savefig('plots/'+name, dpi=100)
    plt.close()

In [9]:
def getthemean(filename,colver='iteration',whichcol=['hr_top10']):
    data=pd.read_csv(filename)
    num=int(np.max(data[colver]))
    meandata=data[data[colver]==1][whichcol].get_values()
    for i in range(2,num+1):
        meandata+=data[data[colver]==i][whichcol].get_values()
    meandata/=num
    return meandata

In [10]:
schemes1=['express-','greedy-','TS-','TSe4-']
schemes2=['TS-','TSe2d2-','TSe2d4-','TSe2d0-','express-']
schemes3=['TS-','TSe4d2-','TSe4-','TSe4d0-','express-']
schemes4=['TS-','TSe4-','newTSthres-','newTSregthres-']
schemes5=['TSe4-','greedy-','TS-','newTSthres-']
schemes6=['express-','winapprox-','winapproxthres-']
schemes7=['express-','TS-','TSe4-','newTSthres-']
schemes8=['express-','newTSthres-','uncert-','mismin-']
schemes9=['winapproxthres-','newTSregthres-','greedythres-']

In [14]:

histplotsshort(name='hr120v20k3.pdf',design='v20',k=3,schemes=schemes1,ylim=[.5,1])


['bhist-fixed_express-120v20.csv', 'bhist-greedy-120v20.csv', 'bhist-TS-120v20.csv', 'bhist-TSe4-120v20.csv']


In [15]:

histplotsshort(name='hr120v20k10e2.pdf',design='v20',k=10,schemes=schemes2,ylim=[.5,1])


['bhist-fixed_express-120v20.csv', 'bhist-TS-120v20.csv', 'bhist-TSe2d0-120v20.csv', 'bhist-TSe2d2-120v20.csv', 'bhist-TSe2d4-120v20.csv']


In [16]:

histplotsshort(name='hr120v20k10e4.pdf',design='v20',k=10,schemes=schemes3,ylim=[.5,1])

['bhist-fixed_express-120v20.csv', 'bhist-TS-120v20.csv', 'bhist-TSe4-120v20.csv', 'bhist-TSe4d0-120v20.csv', 'bhist-TSe4d2-120v20.csv']


In [30]:

histplotsshort(name='hr120v20k10thres.pdf',design='v20',k=10,schemes=schemes4,ylim=[.5,1])
histplotsshort(name='hr120v20k20thres.pdf',design='v20',k=20,schemes=schemes4,ylim=[.5,1])

['bhist-newTSregthres-120v20k10.csv', 'bhist-newTSthres-120v20k10.csv', 'bhist-TS-120v20.csv', 'bhist-TSe4-120v20.csv']
['bhist-newTSregthres-120v20k20.csv', 'bhist-newTSthres-120v20k20.csv', 'bhist-TS-120v20.csv', 'bhist-TSe4-120v20.csv']


In [29]:

#histplotsshort(name='hr120v20k10thres.pdf',design='v20',k=10,schemes=schemes4,ylim=[.5,1])
histplotsshort(name='hr120v20k20t.pdf',design='v20',k=20,schemes=schemes5,ylim=[.5,1])

['bhist-greedy-120v20.csv', 'bhist-newTSthres-120v20k20.csv', 'bhist-TS-120v20.csv', 'bhist-TSe4-120v20.csv']


In [44]:

#histplotsshort(name='hr120v20k10thres.pdf',design='v20',k=10,schemes=schemes4,ylim=[.5,1])
histplotsshort(name='hr120v20k20t2.pdf',design='v20',k=20,schemes=schemes6,ylim=[.5,1])

['bhist-fixed_express-120v20.csv', 'bhist-winapprox-120v20.csv', 'bhist-winapproxthres-120v20k20.csv']


In [47]:

#histplotsshort(name='hr120v20k10thres.pdf',design='v20',k=10,schemes=schemes4,ylim=[.5,1])
histplotsshort(name='hr120v20k3TS.pdf',design='v20',k=3,schemes=schemes7,ylim=[.5,1])
histplotsshort(name='hr120v20k10TS.pdf',design='v20',k=10,schemes=schemes7,ylim=[.5,1])

['bhist-fixed_express-120v20.csv', 'bhist-newTSthres-120v20k3.csv', 'bhist-TS-120v20.csv', 'bhist-TSe4-120v20.csv']
['bhist-fixed_express-120v20.csv', 'bhist-newTSthres-120v20k10.csv', 'bhist-TS-120v20.csv', 'bhist-TSe4-120v20.csv']


In [52]:

histplotsshort(name='hr120v20k3.pdf',design='v20',k=3,schemes=schemes8,ylim=[.5,1])
histplotsshort(name='hr120v20k10.pdf',design='v20',k=10,schemes=schemes8,ylim=[.5,1])
histplotsshort(name='hr120v20k20.pdf',design='v20',k=20,schemes=schemes8,ylim=[.5,1])
histplotsshort(name='hr120v20k40.pdf',design='v20',k=40,schemes=schemes8,ylim=[.5,1])

['bhist-fixed_express-120v20.csv', 'bhist-mismin-120v20k3.csv', 'bhist-newTSthres-120v20k3.csv', 'bhist-uncert-120v20k3.csv']
['bhist-fixed_express-120v20.csv', 'bhist-mismin-120v20k10.csv', 'bhist-newTSthres-120v20k10.csv', 'bhist-uncert-120v20k10.csv']
['bhist-fixed_express-120v20.csv', 'bhist-mismin-120v20k20.csv', 'bhist-newTSthres-120v20k20.csv', 'bhist-uncert-120v20k20.csv']
['bhist-fixed_express-120v20.csv', 'bhist-mismin-120v20k40.csv', 'bhist-newTSthres-120v20k40.csv', 'bhist-uncert-120v20k40.csv']


In [55]:
histplotsPTUshort(name='PTU120v20k20.pdf',design='v20',k=20,schemes=schemes7)

['bhist-fixed_express-120v20.csv', 'bhist-newTSthres-120v20k20.csv', 'bhist-TS-120v20.csv', 'bhist-TSe4-120v20.csv']


In [36]:
histplotsshort(name='hr35v15k3.pdf',items='35',design='v15',k=3,schemes=schemes5+schemes6+schemes8,ylim=[.5,1],end=49)
histplotsshort(name='hr35v15k10.pdf',items='35',design='v15',k=10,schemes=schemes8+schemes9,ylim=[.5,1],end=49)

['bhist-fixed_express-35v15.csv', 'bhist-greedy-35v15.csv', 'bhist-mismin-35v15k3.csv', 'bhist-TS-35v15.csv', 'bhist-TSe4-35v15.csv', 'bhist-uncert-35v15k3.csv', 'bhist-winapprox-35v15.csv']
['bhist-fixed_express-35v15.csv', 'bhist-greedythres-35v15k10.csv', 'bhist-mismin-35v15k10.csv', 'bhist-newTSregthres-35v15k10.csv', 'bhist-newTSthres-35v15k10.csv', 'bhist-uncert-35v15k10.csv', 'bhist-winapproxthres-35v15k10.csv']


In [14]:
histplotsshort(name='hr120v20k10check.pdf',design='v20',k=10,schemes=schemes6+['newTSregthres']+['greedythres'],ylim=[.5,1])

['bhist-fixed_express-120v20.csv', 'bhist-greedythres-120v20k10.csv', 'bhist-newTSregthres-120v20k10.csv', 'bhist-winapprox-120v20.csv', 'bhist-winapproxthres-120v20k10.csv']


In [20]:
schemes20=['TSapprox-','TSapproxthres-','misminapprox-','mismin-','winmismin-','winapprox-','winapproxthres-']
histplotsshort(name='hr120v20k10apporx.pdf',design='v20',k=10,schemes=schemes20,ylim=[.5,1])
histplotsshort(name='hr120v20k20apporx.pdf',design='v20',k=20,schemes=schemes20,ylim=[.5,1])

['bhist-mismin-120v20k10.csv', 'bhist-misminapprox-120v20k10.csv', 'bhist-TSapprox-120v20.csv', 'bhist-TSapproxthres-120v20k10.csv', 'bhist-winapprox-120v20.csv', 'bhist-winapproxthres-120v20k10.csv', 'bhist-winmismin-120v20k10.csv']
['bhist-mismin-120v20k20.csv', 'bhist-misminapprox-120v20k20.csv', 'bhist-TSapprox-120v20.csv', 'bhist-winapprox-120v20.csv', 'bhist-winapproxthres-120v20k20.csv', 'bhist-winmismin-120v20k20.csv']


In [66]:
schemes7=['express-','TS-','TSe4-','greedy-']
schemes8=['express-','newTSthres-','uncert-','mismin-']
histplotsshort(name='hr120v20k3mis1.pdf',design='v20',k=3,schemes=schemes7,ylim=[0,.7],mis=1)
histplotsshort(name='hr120v20k3mis2.pdf',design='v20',k=3,schemes=schemes8,ylim=[0,.7],mis=1)

['bhist-fixed_express-120v20k3-mis.csv', 'bhist-greedy-120v20-mis.csv', 'bhist-TS-120v20-mis.csv', 'bhist-TSe4-120v20-mis.csv']
['bhist-fixed_express-120v20k3-mis.csv', 'bhist-mismin-120v20k3-mis.csv', 'bhist-newTSthres-120v20k3-mis.csv', 'bhist-uncert-120v20k3-mis.csv']


In [67]:
schemes7=['express-','TS-','TSe4-','greedy-']
schemes8=['express-','newTSthres-','uncert-','mismin-']
histplotsshort(name='hr120v20k3mis1top5.pdf',design='v20',k=3,schemes=schemes7,ylim=[0,1],mis=1,low=5)
histplotsshort(name='hr120v20k3mis2top5.pdf',design='v20',k=3,schemes=schemes8,ylim=[0,1],mis=1,low=5)

['bhist-fixed_express-120v20k3-mis.csv', 'bhist-greedy-120v20-mis.csv', 'bhist-TS-120v20-mis.csv', 'bhist-TSe4-120v20-mis.csv']
['bhist-fixed_express-120v20k3-mis.csv', 'bhist-mismin-120v20k3-mis.csv', 'bhist-newTSthres-120v20k3-mis.csv', 'bhist-uncert-120v20k3-mis.csv']


In [68]:
schemes7=['express-','TS-','TSe4-','greedy-']
schemes8=['express-','newTSthres-','uncert-','mismin-']
histplotsshort(name='hr120v20k3moremis1.pdf',design='v20',k=3,schemes=schemes7,ylim=[0,.2],mis=2)
histplotsshort(name='hr120v20k3moremis2.pdf',design='v20',k=3,schemes=schemes8,ylim=[0,.2],mis=2)

['bhist-fixed_express-120v20-moremis.csv', 'bhist-greedy-120v20-moremis.csv', 'bhist-TS-120v20-moremis.csv', 'bhist-TSe4-120v20-moremis.csv']
['bhist-fixed_express-120v20-moremis.csv', 'bhist-mismin-120v20k3-moremis.csv']


In [35]:
schemes7=['express-','TS-','TSe4-','greedy-']
schemes8=['express-','newTSthres-','uncert-','mismin-']
histplotsshort(name='hr120v20k3moremis1top10.pdf',design='v20',k=3,schemes=schemes7,ylim=[0,1],mis=2,low=10)
histplotsshort(name='hr120v20k3moremis2top10.pdf',design='v20',k=3,schemes=schemes8,ylim=[0,1],mis=2,low=10)

['bhist-fixed_express-120v20-moremis.csv', 'bhist-greedy-120v20-moremis.csv', 'bhist-TS-120v20-moremis.csv', 'bhist-TSe4-120v20-moremis.csv']
['bhist-fixed_express-120v20-moremis.csv', 'bhist-mismin-120v20k3-moremis.csv']
